In [1]:
CONSTRAINTS={
    "calories":{
        "min":200,
        "max":300
    },
    # "proteins":{
    #     "min":20,
    #     "max":50
    # },
    # "fats":{
    #     "min":10,
    #     "max":30
    # },
}

INGREDIENTS=["milk","sugar","honey","bread"]



In [2]:
import os
import re
from convex_optimisation import Optimisation
from conversational_agent import LLM
from ratatouille_model_parser import RatatouilleModel

import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

/home/vinayak21574/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-01 02:40:26.953723: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-01 02:40:27.473781: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
! export HUGGINGFACE_API_KEY=<YOUR_TOKEN_HERE>

In [4]:
here=RatatouilleModel()
RECIPE=here.generate_recipe(INGREDIENTS)

In [5]:
model=LLM("meta-llama/Llama-3.1-8B-Instruct")

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.09s/it]


In [6]:
task=None
# unique_ingred=None

In [ ]:
with open("./resources/prompt_phrase2gram.txt","r") as f:
    task=f.read()


In [8]:
res=model.chat(message=RECIPE["ingredients"],persona=task)

In [9]:
pattern = r'\(([^,]+), ([^)]+)\)'

matches = re.findall(pattern, res)

result = [(item.strip(), weight.strip()) for item, weight in matches]


In [10]:
INGRED=[]
QTY=[]

for a,b in result:
    INGRED.append(a)
    QTY.append(max(float(b),1))

In [11]:
here=Optimisation(INGRED,QTY,CONSTRAINTS)

In [12]:
qty,nutrient_profile=here.fit()


In [13]:
QUANTITIES=[]

for name,q in zip(INGRED,qty):
    QUANTITIES.append((name,q))

In [14]:
RECIPE["title"]

'Pears Congri Traditional'

In [15]:
QUANTITIES

[('pear', 47.04),
 ('salt', 0.16),
 ('water', 7.06),
 ('sugar', 3.14),
 ('egg', 0.16),
 ('honey', 0.16),
 ('vanilla extract', 0.08),
 ('flour', 19.6),
 ('sugar', 10.5),
 ('egg yolks', 0.47),
 ('flour', 19.6),
 ('demerara sugar', 7.06),
 ('salt', 0.04),
 ('pepper', 0.02)]

In [16]:
RECIPE["instructions"]

['Preheat oven to 200c',
 'Cut the pears into quarters and sprinkle with salt',
 'Place in oven and bake until just tender and the pears are soft enough to break apart with a fork, approximately 30 to 40 minutes',
 'Remove from oven and cool slightly',
 'Remove the pears from the oven and let cool a little before opening the pears to remove core',
 'Cut each quarter into 4 equal pieces',
 'Set aside',
 'Using a mortar and pestle, mash the salt and sugar together',
 'Add the egg, honey and vanilla extract and mix together',
 'Add the flour and mix',
 'Add the pears and mix gently',
 'Add the flour gradually, mixing well until the mixture forms into a smooth dough',
 'Using hands or your hands, form the dough into cylinders about 2 cm in diameter and 8 cm in height',
 'Place on a greased baking sheet',
 'Bake in the preheated oven until light golden brown, approximately 10 to 12 minutes, rotating the tray halfway through baking',
 'Set aside and allow to cool before serving',
 'To make t

In [17]:
nutrient_profile

{'calories': 250.06, 'proteins': 4.24, 'fats': 0.47}